In [1]:
from aux_func_V3 import *
from aux_gif2D import *
import API_Numpy
import API_TensorFlow

import dill
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
API=API_TensorFlow
# API=API_Numpy

ε=1e-40
WENOs_list=[
    'WENO-Z',
    'WENO-Z (MS)',
    'WENO-Z (BI)',
    'WENO-JS (BI)',
    'WENO-ZC (BI)'
    ]

In [3]:
γ = 5.0/3.0
N = 800
x, y, Δx, Δy, U0 = CondiçãoInicialRayleighTaylor_N(N, γ)
cfl = 0.3
t_final = 1.95

GhostPointsX=RayleighTaylorGhostPointsX()
GhostPointsY=RayleighTaylorGhostPointsY(γ,dtype=dtype)

WENOs={}

for name in WENOs_list:
    WENOs[name]=WENO_dict[name+' 2D'](API,
                            euler_equation_2D,
                            γ=γ)

In [4]:
N

800

In [4]:
for name in WENOs_list:
    create_data(
        'RT',
        N,
        name,
        U0,
        WENOs[name],
        0.01,
        t_final,
        cfl,
        Δx,
        Δy,
        GhostPointsX,
        GhostPointsY,
        RayleighTaylorGravity,
        continue_flag=True)

    create_movie(
        'RT',
        N,
        name,
        x,
        y,
        figsize=(10,16),
        vmin=0.9,
        vmax=2.3,
        levels=15,
        xlim=None,
        ylim=(0.2,0.8))

WENO-Z
Tempo inicial: 1.95
Tempo final: 1.95
WENO-Z (MS)                      
Tempo inicial: 1.95
Tempo final: 1.95
WENO-Z (BI)                      
Tempo inicial: 0.8300000000000001
Tempo final: 1.950000000000001                      
WENO-JS (BI)                     
Tempo inicial: 0.0
Tempo final: 1.9500000000000015                       
WENO-ZC (BI)                     
Tempo inicial: 0.0
Tempo final: 1.9500000000000015                       


In [3]:
WENOs_list=[
    # 'WENO-JS',
    # 'WENO-JS (BI)',
    'WENO-Z',
    'WENO-Z (MS)',
    # 'WENO-Z (BI)',
    # 'WENO-Zp',
    # 'WENO-ZC (BI)'
    ]

In [8]:
for N in (200,):
    γ = 7.0/5.0
    #N = 200
    x0=1/6
    x, y, Δx, Δy, U0 = CondiçãoInicialDoubleMach_N(N, γ,x0)
    cfl = 0.45
    t_final = 0.2

    label=f'DoubleMach'

    GhostPointsX=DoubleMachGhostPointsX(L=3,R=3,γ=γ)
    GhostPointsY=DoubleMachGhostPointsY(L=3,R=3,γ=γ,x0=x0,x=x)

    WENOs={}

    for name in WENOs_list:
        WENOs[name]=WENO_dict[name+' 2D'](API,
                                euler_equation_2D,
                                γ=γ)

    for name in WENOs_list:
        create_data(
            label,
            N,
            name,
            U0,
            WENOs[name],
            0.001,
            t_final,
            cfl,
            Δx,
            Δy,
            GhostPointsX,
            GhostPointsY,
            NullForce,
            continue_flag=True)
        create_movie(
            label,
            N,
            name,
            x,
            y,
            save_dir=label+'Zoomed',
            figsize=(12,9),
            vmin=2,
            vmax=20,
            levels=100,
            xlim=(2.0,3.0),
            ylim=(0,0.6),
            colorbar=False,
            start_time=130)
        create_movie(
            label,
            N,
            name,
            x,
            y,
            figsize=(3*8,8),
            vmin=1,
            vmax=22,
            levels=100,
            xlim=(0,3),
            ylim=None)

In [4]:
for N in (200,):
    γ = 7.0/5.0
    #N = 200
    x0=1/6
    x, y, Δx, Δy, U0 = CondiçãoInicialDoubleMach_N(N, γ,x0)
    cfl = 1
    t_final = 0.25

    label=f'DoubleMach-CFL={cfl}'

    GhostPointsX=DoubleMachGhostPointsX(L=3,R=3,γ=γ)
    GhostPointsY=DoubleMachGhostPointsY(L=3,R=3,γ=γ,x0=x0,x=x)

    WENOs={}

    for name in WENOs_list:
        WENOs[name]=WENO_dict[name+' 2D'](API,
                                euler_equation_2D,
                                γ=γ)

    for name in WENOs_list:
        create_data(
            label,
            N,
            name,
            U0,
            WENOs[name],
            0.001,
            t_final,
            cfl,
            Δx,
            Δy,
            GhostPointsX,
            GhostPointsY,
            NullForce,
            continue_flag=True)

        create_movie(
            label,
            N,
            name,
            x,
            y,
            figsize=(4*8,8),
            vmin=1,
            vmax=22,
            levels=100,
            xlim=(0,4),
            ylim=None)
        create_movie(
            label,
            N,
            name,
            x,
            y,
            save_dir=label+'Zoomed',
            figsize=(12,9),
            vmin=2,
            vmax=20,
            levels=100,
            xlim=(2.6,3.4),
            ylim=(0,0.6),
            colorbar=False,
            start_time=180)

WENO-Z
Tempo inicial: 0.25
Tempo final: 0.25
WENO-Z (MS)                      
Tempo inicial: 0.25
Tempo final: 0.25


In [5]:
create_movie(
        'DoubleMachZoom',
        N,
        'WENO-JS (BI)',
        x,
        y,
        figsize=(8,8),
        vmin=1,
        vmax=18,
        levels=100,
        xlim=(2,3),
        ylim=(0,0.5),
        colorbar=False)

In [ ]:
γ = 7.0/5.0
θ = API_Numpy.pi/6.0
N = 400
x0=1/6
x, y, Δx, Δy, U0 = CondiçãoInicialShockEntropy2D_N(N, γ,θ)
cfl = 0.5
t_final = 1.8

GhostPointsX=ShockEntropy2DGhostPointsX(γ,Δx,y,θ)
GhostPointsY=ShockEntropy2DGhostPointsY()

WENOs={}

for name in WENOs_list:
    WENOs[name]=WENO_dict[name+' 2D'](API,
                            euler_equation_2D,
                            γ=γ)

In [ ]:
for name in WENOs_list:
    create_data(
        'ShockEntropy',
        N,
        name,
        U0,
        WENOs[name],
        0.01,
        t_final,
        cfl,
        Δx,
        Δy,
        GhostPointsX,
        GhostPointsY,
        NullForce,
        continue_flag=True)

    create_movie(
        'ShockEntropy',
        N,
        name,
        x,
        y,
        figsize=(2*8,8),
        vmin=0.75,
        vmax=4.75,
        levels=15,
        xlim=None,
        ylim=None)

WENO-Z (BI)
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Z                           
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Zp                          
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Z (MS)                      
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Zp (MS)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-Zp (BI)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-JS (BI)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-ZC (MS)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
WENO-ZC (BI)                     
Tempo inicial: 0.0
Tempo final: 1.8000000000000014                       
